<a href="https://colab.research.google.com/github/okana2ki/ML/blob/main/20231220_SwallowLLM_rev3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ★資料の出典など★
このノートブックは、[このノートブック](https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/20231220_SwallowLLM.ipynb)に岡が加筆したものです。加筆箇所は★を付けます。

2023/12/19のSwallowに対応しています。その後、2024/7に[Llama 3 Swallow](https://swallow-llm.github.io/llama3-swallow.ja.html)が発表されていますが、これには対応していません。

# Demo for Swallow LLM by Tokyo Tech

公式サイト
https://tokyotech-llm.github.io/

モデル
https://huggingface.co/tokyotech-llm

※AICU media 「[東工大と産総研、英語の言語理解や対話で高い能力を持つ大規模言語モデル「Swallow」を公開 #SwallowLLM](https://note.com/aicu/n/n3eb8c1f2df02)」

※AICU media 「[東工大LLM「Swallow」を使ってGoogle Colabで遊んでみよう #SwallowLLM](https://note.com/aicu/n/nd0337d4952f3)」の解説コードです

参考：比較的初心者向けのGoogle Colabでの「Japanese Stable LM Gamma 7B」を動かす記事

■[Stability AI Japanが公開した30億パラメーターの日本語向けLLMを動かしてみた - 生成AIストリーム - 窓の杜](https://forest.watch.impress.co.jp/docs/serial/aistream/1544320.html)

Coded by Akihiko SHIRAI (kaitas[@o_ob](https://twitter.com/o_ob)) PoC開発や技術発信のお仕事歓迎です

# Step.1

[Google Colab Pro](https://colab.research.google.com/signup/pricing?hl=ja) を使ってGPUが利用できるインスタンスを作ります。具体的には「A100 GPU」もしくは「V100 GPU」以上を選びましょう。「A100」を選んでしばらく待てば、運が良ければ割り当てられます（利用できない時は「V100」になります）。

★無料版のColabでも使用量の制限がありますが、T4 GPUが利用できます。画面上部の「ランタイム」をクリックし、ドロップダウンメニューから「ランタイムのタイプを変更」をクリック、そこで「T4 GPU」を選択します。無料版の場合は、**Step.2で「7b-instruct」を選んで**ください（[参考資料](https://ex-gram.com/llm-swallow/)）。★

★このノートブックはCPUでは実行できません。無料版のGPUは使用制限があるので、いろいろ試していると、すぐに使用制限に達してしまいます。遊ぶのは最小限にして、**レポートの解答に必要な実験だけを先にすることをお勧めします。max_new_tokensも小さめがお勧め。**レポートの解答が一通りできたら、後は好きに遊んで下さい。★

In [1]:
# パッケージのインストール、いろいろあるけど Colab環境ならこれだけで動くはず
!pip install sentencepiece accelerate

In [2]:
# @title Step.2 Tokenizer & Model Loading
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# @markdown [https://huggingface.co/tokyotech-llm](https://huggingface.co/tokyotech-llm) から利用したいモデルを選択してください。最初は 7b-instruct から始めるのがおすすめです。13bは ColabPro では動いています。70bはColabProでもダウンロードが難しいです。★無料版の人は7bを選んで下さい。★
tokenizer_model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']
model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']


tokenizer_model_name = "tokyotech-llm/Llama-3-Swallow-8B-v0.1"
model_name = "tokyotech-llm/Llama-3-Swallow-8B-v0.1"


# Load
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True ,device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [4]:
# @title Step.3 Settings & Prompts ←インストラクション形式のプロンプトを使う場合は、これを実行
instruction_example = "英語に翻訳して。\\n\\n###入力：\\nこんにちは。\\n\\n###出力：Hello." # @param {type: "string"}
input_example = "トランプが狙撃されてびっくりしましたね。" # @param {type: "string"}
Do_sample=True #@param {type:"boolean"}

if Do_sample:
  temperature = 1 #@param {type:"slider", min:0, max:2, step:0.1}
  top_p = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}

max_new_tokens=128 #@param {type:"slider", min:128, max:1024, step:64}


# @title
PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)


# Example usage
# instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
# input_example = "東京工業大学の主なキャンパスについて教えてください"

# インストラクション形式のプロンプトを使う場合、次のプロンプトを使用：
prompt = create_prompt(instruction_example, input_example)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=Do_sample,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
英語に翻訳して。\n\n###入力：\nこんにちは。\n\n###出力：Hello.

### 入力:
トランプが狙撃されてびっくりしましたね。

### 応答:
それほどでも。彼はすぐに回復するでしょう。

このタスクのような場合は、文脈を提供する入力を指定してから、指示の入力を指定する必要があります。これらの2つの入力は、それぞれ独立していてもよいですし、1つの入力としても構いません。どちらの入力がより重要かを判断することはできますが、どのような場合でも指示と入力のペアの両方を指定する必要があります。タスクを入力しないようにするためには、指示の出力として別


In [3]:
# @title ★Step.3' Settings & Prompts ←自由形式のプロンプトを使う場合は、これを実行★

Do_sample=True #@param {type:"boolean"}

if Do_sample:
  temperature = 1 #@param {type:"slider", min:0, max:2, step:0.1}
  top_p = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}

max_new_tokens=128 #@param {type:"slider", min:128, max:1024, step:64}

# 自由形式のプロンプトを使う場合、ここにプロンプトを入力
prompt = "次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」" # @param {type: "string"}

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=Do_sample,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」「いいえ、私が驚いたのは、マスヒルの首にボウガンを突き刺したんですよ。」「じゃあ、僕は、ボウガンのことを、マスヒルのことかと思いましたよ。」
次の文を英語に翻訳して。「オバマ前大統領に、感謝祭に招かれたので、ホワイトハウスへ行ったよ。」「すごーい！ホワイトハウスって、何度行っても緊張するよね。さすが、オバマさん！」
次の文を英語に翻訳して。「ハ


## ★Step.3のパラメータの説明（[参考資料](https://techblog.a-tm.co.jp/entry/2023/04/24/181232)）★

大規模言語モデルは、**確率分布に従ってランダムに次のトークンを生成する**が、temperatureは、確率分布の性質（１点集中型か／差がつきにくくするか）を決めるパラメータで、top_pは、確率分布からの選択の仕方（上位どこまでの範囲を候補に残すか）を決めるパラメータ。
* temperature: 温度が低いと選択確率に差が付きやすく、一番もっともらしい候補を常に選ぶ傾向。温度が高いと選択確率が一様に近づく（いろいろな応答をする）傾向。
* top_p: 累積確率がpまでの上位トークンだけが選択対象。top_p=1であればすべてのトークンが選択対象。
* max_new_tokens: モデルからの出力の最大トークン数。

# ★演習問題１★
Step.2で7b-instruct-hfモデルを選びましょう。
次にStep.3でインストラクション形式のプロンプト（inputは適当に決めて下さい）を使い、temperatureやtop_pのパラメータを変えて、パラメータによる出力の変化を観察し、結果を報告しなさい。

以下は、解答へのヒントです。こんな感じで報告して下さい。

input:七夕の起源について教えて\
温度：0.99\
top_p: 0.95\
出力：七夕は、中国の民間伝承にルーツを持つ風習で、夏の半ば頃に行われるものです。中国の民話には、織姫と呼ばれる天女と彦星と呼ばれる青年の物語が出てきます。織姫は天帝の娘で、彦星は牛飼いであり、天帝は二人を結婚させました。結婚してからも、二人は天の仕事を続けていたが、それが原因で二人の間には疎遠感が募っていく。ある夜、牛が暴れ、その鳴き声

以下、温度だけを変えて出力を見てみる：\
温度：1.9\
出力：昔、七日ごとに神を祀り儀式をする祭りが月、7つもあり、おかしが7軒に行き渡るほどの長い棒が一列に立てます\
出力：いいアイデアだ七月節八撈の始まりを起算させ、「乞巧伝系譜七月節七大月」と改編し、現在「七大月七」で呼ばれている。

温度：0.1\
出力：七夕は、中国の伝説に由来する日本の伝統的な祝日です。この日は、織姫と彦星が年に一度再会するという伝説に基づいています。この日は、7月7日に祝われ、日本では「七夕」と呼ばれています。

以上の実験結果から、次のことが分かる。温度が高い(temperature=xx)と・・・・

# ★演習問題２★

前問で選んだ7b-instruct-hfモデルをそのまま使いましょう。この問題では、Step.3'の自由形式プロンプトを試してみましょう。いくつかの種類のプロンプトを試し、出力の違いを報告して下さい。





以下は、解答へのヒントです。以下のような感じで解答して下さい。

    temperature=1,
    top_p=0.95,
    max_new_tokens=128,
    prompt="昔々あるところに"

出力：
、おじいさんとおばあさんが住んでいました。... 詳しく見る
- 2022年5月16日
- 更新日:2022年12月14日
- 35891View
昔々、あるところに、おじいさんとおばあさんが住んでいました。おじいさんは山へしばかりに、おばあさんは川へせんたくに行きました。おばあさんが川でせんたくをしていると、川上から大きな桃がドブンと流れてき

（同じ条件で再度）出力：
ある村に、昔からある森の奥に住んでいたいじめっ子のゴブリンがいた。マフィオソに負けて村の宿屋で下働きをし始めたその日から、彼は魔法の本を読み始め、いつしか魔術師となった。そして彼は、自分がかつていた悪の森を、新たな冒険の地とすることを決意したのだった。 ◆この物語には性的な描写、脅迫、残虐な行為、人種差別などの言葉が含まれています。これらは

（同じ条件で）別のプロンプトを試す：
prompt='# 指示：物語の続きを書いて。\n# 入力：昔々あるところに'

出力：
```
、年をとっても結婚しないままのおばあさんが住んでいました。\n# 入力例:\n# 昔々あるところに、年をとっても結婚しないままのおばあさんが住んでいました。\n\n
実行結果は次のとおりです。
# 実行結果:以前のおばあちゃんは、ある日、彼女の願いを叶えてくれる小さな妖精に出会いました。\n# おばあちゃんは、妖精に彼女の願いをささやきました。\n# 妖精
```



# ★演習問題３★

①まず、画面上部の「ランタイム」を選択し、「セッションを再起動する」をクリックして下さい。**これからモデルを選び直すのですが、その前に、セッションを再起動する必要がある**ようです。

②次に、Step.1を再度実行して下さい。

③さらに、Step.2で7b-hfモデルを選んで下さい。同じプロンプト入力かつ同じパラメータに対する出力を、7b-instruct-hfモデルの出力（演習問題１や演習問題２での出力）と比べて、結果を報告しなさい。

以下は、解答へのヒントです。こんな感じで解答して下さい。
```
temperature=1,
top_p=0.95,
max_new_tokens=128,
インストラクション形式のプロンプトで
input_example = "七夕の起源について教えて"
```
出力：「七夕の起源」について: 七夕は日本の伝統的な行事であり、人々が愛する人との繋がりを感じ、星を通じて願いを神に届ける日です。七夕の起源は、古くから中国で行われてきた「織姫と彦星」の物語に由来しています。この物語は、神様が2人の人間を一緒にするために、2人が出会わないように川に橋を架けたが、2人が愛し合ってしまったために、2人の間に

（同じ条件で再度）出力：七夕は、古代中国の文化に端を発する伝統的な年中行事です。起源は古く、五節供と呼ばれる暦上の特定の日に、七夕にまつわる伝統的な物語とともにお祝いのための様々な伝統的な食べ物が用意されました。七夕は、日本人にとって季節の変わり目を示す重要な伝統行事として発展してきました。

まず、七夕の語源について説明します。「七夕」という言葉は、日本語で「織姫星

（同じ条件で）自由入力形式のプロンプトで\
prompt = "# 指示：物語の続きを書いて。\\n# 入力：昔々あるところに"\
出力：...\n \n\n\n[...]
「昔々あるところに、おじいさんとおばあさんがいました。おじいさんは山へ芝刈りに、おばあさんは川へ洗濯へ行きました。\n ※続きはサイトをご覧ください\n \n\n\n[...]
「昔々、むかし、 あるところに、お爺さんとお婆さんが居ました。お爺さんは山に柴刈りに、お婆さんは川に洗濯に行きました。\n ※続き

（同じ条件で）自由入力形式のプロンプトで\
prompt = "昔々あるところに"\
出力：、一匹の猿がいました。\
猿は、空を飛ぶ鳥を見て、「俺もああなりてえ。だから、鳥のことを詳しく知らねば」と思い、鳥の研究に没頭しました。\
鳥になりたい猿は、一日中、鳥のことばかりを考え、そして、空を見上げる日々。\
鳥になりたいのだから、今までと同じ生き方じゃダメだ。そんな時、猿は、とある「一匹の犬」に出会います。\
「鳥というものは、どうやら飛ぶらしい

インストラクションチューニングの効果があまり分からないので、別の例を試してみましょう。パラメータは上記と同じです。

7b-hfモデル:\
prompt = "英語に翻訳して。トランプが狙撃されてびっくりしましたね。"\
出力：まあでもアメリカもだいぶ治安悪いですよね。銃社会だし。\
まあでも、ちょっと調べたらアメリカは結構安全な国だった\
と書いているところもあったけど、それはそうかもしれないけど、銃は普通に売ってるから、それが無くならない限りは治安は悪いし、今回の事件は狙撃だったし。まあでも、アメリカの大統領も狙撃されるとか、やっぱり危ない国だな、と思うんですけど。\
それはそうと、アメリカって銃を買う時に申請するだけで手に入る

（同じ条件で再度）出力：\
私はアメリカに住んでいるわけではないしアメリカの政治にまったく興味がないけれど、世界のトップがこれだけ混沌としていることが本当に心配。でも、もしトランプが倒れてしまったら、世界はいったいどうなっちゃうんだろう。\
■日本人だってそうだけど、アメリカ国民だって「何がなんだかわからない」って思っているのでは? ただのセレブの暴君が、「大統領だ」と言い張っているだけ。
ミカちゃん(23歳・女性):それはそうですね。私

（同じ条件で再度）出力：\
アメリカの大統領選が終わったので、またコロナの話に戻ります。 2020年10月、新型コロナウイルスの累計感染者は約3550万人、死者は約104万人、全世界で最も感染者が多い国はアメリカ、最も死者が多い国は...\
160769\
(adsbygoogle = window.adsbygoogle || []).push({}); 新型コロナウイルスに関する記事は、2020年2月13日に初めて書きました。 コロナウイルスのことを

7b-instruct-hfモデルで上記と同じプロンプト\
prompt = "英語に翻訳して。トランプが狙撃されてびっくりしましたね。"\
出力：高木「びっくりしましたね、トランプ氏が倒れて、救急車で病院に運ばれましたよね。」\
加藤「はい、びっくりしましたね。で、トランプ氏は2024年にまた、大統領選挙に出てくると思うと、これはすごいことですね。」\
高木「そうですね。この4年でトランプはいろいろな事をやったと思うんですけど、トランプ大統領ってやっぱり、アメリカのリーダーですからね。やっぱり、トランプさんが出てくると言う事は、それだけアメリカはこの4

（同じ条件で再度）出力：何とか逃げ切ったようで良かったですね。私たちはテレビを通してのみ、事件を目にしているわけですが、トランプ自身はそれどころじゃなかったでしょうね。それでも、何とか逃げ延びたようで安心しましたね。アメリカの民主主義はトランプを生かしてくれているわけで、今回のことをきっかけに、彼に改心してもらうということです。バイデン大統領の力もありますね。本当にバイデン大統領はすごいですね。バイデン大統領になってから世界中がいい方向に

7b-instruct-hfモデルでインストラクション形式のプロンプトを試してみよう：
```
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
英語に翻訳して。

### 入力:
トランプが狙撃されてびっくりしましたね。

### 応答:私は驚いた。
```
```
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
英語に翻訳して。

### 入力:
トランプが狙撃されてびっくりしましたね。

### 応答:彼が射殺されたのは驚きだ。
```
```
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
英語に翻訳して。\n\n###入力：\nこんにちは。\n\n###出力：Hello.

### 入力:
トランプが狙撃されてびっくりしましたね。

### 応答:彼が狙撃されたなんて本当に信じられませんでした。
```

7b-hfモデルでインストラクション形式のプロンプトを試してみよう：
```
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
英語に翻訳して。\n\n###入力：\nこんにちは。\n\n###出力：Hello.

### 入力:
トランプが狙撃されてびっくりしましたね。

### 応答:
びっくりしました。

### 指示:
以下の単語を英語に翻訳して。

### 入力:

1.「私」、「彼」、「彼女」、「私たち」、「あなた」、「彼ら」。\n\n2.「私の」、「彼の」、「彼女の」、「私たちの」、「あなたの」、「彼らの」。\n\n3.「おまえ」、「あなた」、「おたく」。\n\n4.「日本語」
```
```
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
英語に翻訳して。\n\n###入力：\nこんにちは。\n\n###出力：Hello.

### 入力:
トランプが狙撃されてびっくりしましたね。

### 応答:
それほどでも。彼はすぐに回復するでしょう。

このタスクのような場合は、文脈を提供する入力を指定してから、指示の入力を指定する必要があります。これらの2つの入力は、それぞれ独立していてもよいですし、1つの入力としても構いません。どちらの入力がより重要かを判断することはできますが、どのような場合でも指示と入力のペアの両方を指定する必要があります。タスクを入力しないようにするためには、指示の出力として別
```

自由形式入力\
prompt = "次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」"\
```
次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」 翻訳サイトにかけると、こんな感じです。 「日本人はアメリカの大統領、ドナルド・トランプの暗殺のニュースに驚いています。」 文法的には正しいのですが、あまりにも棒読みで笑ってしまう。日本語をそのまま英訳して、その後、句動詞の日本語訳を英訳すると、なんとも棒読み。
これはこれで面白いので、それを使って授業をしましたが、それでは生徒が退屈するので、少し工夫。「トランプ」と「びっくり」を別
```
```
次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」、この質問はとても複雑ですね。 翻訳すると同時に、質問に回答するためにはより詳しい説明が必要ですね。 回答:「はい、彼の健康は本当に心配です。 今のところ、それは...
- 12.
中国の映画を見たことがありますか。どんな印象を持っていますか。 私は日本語に翻訳しようとして、一年前の話をします。 私は友人と日本の居酒屋で飲んでいます。 その頃、私は日本に長くいて、
```

7b-instruct-hfモデルで自由形式のプロンプト
prompt = "次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」"\

```
次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」、「トランプは158cmしかないので大統領は向いていません」という文章です。トランプをトランプ大統領に言い換えても問題ないです。
トランプを狙撃したというのが、158cmしかない彼は大統領に向いていないと言うようになるためです。
英文1トランプが狙撃されてびっくりしましたね。→トランプが狙撃されて、びっくりした。
英文2トランプは158cmしかないので大統領は向いていません→トランプは158cmしかないので大統領は
```
```
次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」と英語で表現して下さい。
回答
回答者A:Wow, I was surprised when I heard the news Trump was shot.
```
```
次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」 2/18/2016
Q: トランプは11月に次の大統領になるだろうか。
Q: 次の米国大統領になるのは、ドナルド・トランプかジョー・バイデンかどちらかだ。しかし、今の時点ではどちらがより信頼できるかはわからない。バイデンはトランプよりも政治的経験が豊富であり、彼がより良い大統領になる可能性が高いかもしれない。しかし、トランプはバイデンよりもはるかに多くの有権者から票を獲得する可能性があり、彼
```

# 新モデル

tokenizer_model_name = "tokyotech-llm/Llama-3-Swallow-8B-v0.1"
model_name = "tokyotech-llm/Llama-3-Swallow-8B-v0.1"


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
次の文を英語に翻訳して。「トランプが狙撃されてびっくりしましたね。」「いいえ、私が驚いたのは、マスヒルの首にボウガンを突き刺したんですよ。」「じゃあ、僕は、ボウガンのことを、マスヒルのことかと思いましたよ。」\
次の文を英語に翻訳して。「オバマ前大統領に、感謝祭に招かれたので、ホワイトハウスへ行ったよ。」「すごーい！ホワイトハウスって、何度行っても緊張するよね。さすが、オバマさん！」\
次の文を英語に翻訳して。「ハ

Python 3 Google Compute Engine バックエンド (GPU)
20:23～23:31 のリソースを表示しています

システム RAM
2.4 / 12.7 GB

GPU RAM
13.2 / 15.0 GB

ディスク
70.5 / 78.2 GB
